# Answers in the plots

In [1]:
import re
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from mapper import D, CODE_DICT

### Helpers

In [2]:
years = ("y19", "y20")
non_eng = ('bcom_fm', 'bed', 'bcom_af', 'mms', 'llb', 'bpharm', 'ba', 'bcom_bi')
eng = ('comp', 'chem', 'electronics', 'civil', 'electrical', 'mech', 'it', 'extc')

In [3]:
def get_branch_dict():
    BR = {}
    for y in years:
        for k, i in CODE_DICT.items():
            BR[(y,i)] = k
    BR[('y19', 'mech')] = '1T01417'
    BR[('y20', 'mech')] = '1T01427'
    return BR

def get_concatted_df(d):
    dfs = []
    for y in years:
        for c in eng + non_eng:
            df = getattr(getattr(d, y), c).df.copy()
            for col in ['percent', 'gpa']:
                if col in df: continue
                df[col] = np.nan
            df = df[['name', 'is_female', 'center', 'gpa', 'percent']]
            df['year'] = y
            df['course'] = c
            dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

def get_dc(y, c):
    return getattr(getattr(d, y), c)

def get_row_meta(row):
    dc = get_dc(row['year'], row['course'])
    meta = dc.meta.copy()
    center = dc.center[str(row['center'])]
    meta.update(dict(center=center))
    if '2019' in meta:
        meta.update(dict(date='2019'))
    elif '2020' in meta:
        meta.update(dict(date='2020'))
    else:
        raise Exception('wtf?')
    return meta

### The Main Stuff

In [4]:
d = D()
br = get_branch_dict()
df = get_concatted_df(d)

cleaning and loading data...


done


In [5]:
get_row_meta(df.iloc[18888])

{'code': '1T01227',
 'degree': 'B.E',
 'semester': 'VII',
 'year': None,
 'course': 'INFORMATION TECHNOLOGY',
 'description': 'CHOICE BASED',
 '2019': 'results_dump/S19/1T01227.pdf',
 'center': 'YTEM',
 'date': '2019'}

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148160 entries, 0 to 148159
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   name       148160 non-null  object 
 1   is_female  148160 non-null  int64  
 2   center     148160 non-null  int64  
 3   gpa        138151 non-null  float64
 4   percent    123806 non-null  float64
 5   year       148160 non-null  object 
 6   course     148160 non-null  object 
dtypes: float64(2), int64(2), object(3)
memory usage: 7.9+ MB
